In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from nilearn.input_data import NiftiMasker
from nilearn import datasets, plotting, image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from nilearn.plotting import plot_anat, show, plot_stat_map, plot_matrix

Using TensorFlow backend.


In [2]:
# Designate input file
haxby_ds = datasets.fetch_haxby(subjects=[4], fetch_stimuli=True)

# 'func' is a list of filenames: one for each subject
func_file = haxby_ds.func[0]

# Standardizing
mask_vt_file = haxby_ds.mask_vt[0]
masker = NiftiMasker(mask_img=mask_vt_file, standardize=True)

# Load the behavioral data that I will predict
beh_label = pd.read_csv(haxby_ds.session_target[0], sep=" ")

# Regressout the resting state
nonrest_task_mask = (beh_label['labels'] != 'rest')

#select data
X = masker.fit_transform(func_file)[nonrest_task_mask]
y = beh_label['chunks'][nonrest_task_mask]

# Remove the resting state and find names of remaining active labels
categories = y[nonrest_task_mask].unique()

# Get the labels of the numerical conditions represented by the vector y
unique_conditions, order = np.unique(categories, return_index=True)

# Sort the conditions by the order of appearance
unique_conditions = unique_conditions[np.argsort(order)]

In [3]:
y.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [4]:
#shuffle and split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state = 0)

#standarize features caling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initializing the ANN

In [5]:
classifier = Sequential()

In [6]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 338, init = 'uniform', activation = 'relu', input_dim = 675))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=675, units=338, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [7]:
# Adding the second hidden layer
classifier.add(Dense(output_dim = 338, init = 'uniform', activation = 'relu'))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=338, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [8]:
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [9]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
X_train.shape

(777, 675)

In [11]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, epochs = 10)

Epoch 1/10
777/777 [==============================] - 1s 724us/step - loss: -236.8407 - accuracy: 0.0888
Epoch 2/10
777/777 [==============================] - 0s 185us/step - loss: -4589.0191 - accuracy: 0.0862
Epoch 3/10
777/777 [==============================] - 0s 200us/step - loss: -24425.2982 - accuracy: 0.0862
Epoch 4/10
777/777 [==============================] - 0s 182us/step - loss: -85822.1005 - accuracy: 0.0862
Epoch 5/10
777/777 [==============================] - 0s 183us/step - loss: -217897.0552 - accuracy: 0.0862
Epoch 6/10
777/777 [==============================] - 0s 173us/step - loss: -460704.7884 - accuracy: 0.0862
Epoch 7/10
777/777 [==============================] - 0s 185us/step - loss: -841117.4089 - accuracy: 0.0862
Epoch 8/10
777/777 [==============================] - 0s 179us/step - loss: -1400896.1100 - accuracy: 0.0862
Epoch 9/10
777/777 [==============================] - 0s 217us/step - loss: -2152722.7773 - accuracy: 0.0862
Epoch 10/10
777/777 [============

In [12]:
# Making the predictions and evaluating the model
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [13]:
print(y_pred)

[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]


In [14]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 0  6  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0]]
